In [149]:
import pandas as pd
from utils import strip_tags, spread_choices_by_list
import datetime
import movecolumn as mc 
import numpy as np
import openpyxl
from openpyxl.styles import Alignment
from openpyxl.styles.colors import Color
from openpyxl.styles.fills import PatternFill
from pandas.api.types import is_float_dtype, is_integer_dtype

In [150]:
def getIndexes(df, value):
    # Empty list
    listOfPos = []

    # isin() method will return a dataframe with
    # boolean values, True at the positions   
    # where element exists
    result = df.isin([value])

    # any() method will return
    # a boolean series
    seriesObj = result.any()

    # Get list of column names where
    # element exists
    columnNames = list(seriesObj[seriesObj == True].index)

    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)

        for row in rows:
            listOfPos.append((row, col))

    # This list contains a list tuples with
    # the index of element in the dataframe
    return listOfPos

In [151]:

df_new = pd.read_excel('data\\db_2024_07_26.xlsx')
df_old = pd.read_excel('data\\raw_db_2023_07_29.xlsx')

df_new_orig = df_new.copy()
df_old_orig = df_old.copy()

In [152]:
df_new = df_new_orig.copy()
df_old = df_old_orig.copy()


In [153]:

##### merge faoliyat columns (because they casding-split)
col_change = '5. Корхонанинг асосий иқтисодий фаолият тури:'
df_new[col_change] = df_new[col_change].mask(df_new[col_change] == 'Саноат', df_new['5.1. Саноат фаолият турлари:'])
df_new[col_change] = df_new[col_change].mask(df_new[col_change] == 'Хизмат кўрсатиш', df_new['5.2. Хизмат кўрсатиш соҳалари:'])

df_old['####**5. ТАРМОҚНИ ТАНЛАНГ:**'] = df_old['####**5. ТАРМОҚНИ ТАНЛАНГ:**'].replace({'Қурилиш ишлари': 'Қурилиш',
                                                                                'Бошқа хизматлар30': 'Бошқа хизматлар',
                                                                                'Соғлиқни сақлаш хизматлари30': 'Соғлиқни сақлаш хизматлари',
                                                                                'Қурилиш материаллари20':'Қурилиш материаллари',
                                                                                }, regex=True)

In [154]:
## old ####**23. Сизнингча, солиқ тизимида мавжуд муаммолар нималарда?**

# old_columns = df_old.columns.values.tolist()
# old_columns_edit = []
# for column in old_columns:
#     column = column.replace('Солиқ инспекторлари томонидан олдиндан тўловларни амалга оширишга ундаш', 'Солиқ имтиёзлари сабабли адолат меъзонларининг бузилиши')\
#     .replace('Солиқ имтиёзлари сабабли адолат меъзонларининг бузилиши', 'Солиқ имтиёзлари сабабли адолат мезонларининг бузилиши')\
#     .replace('Ягона солиқ', 'Ягона ижтимоий солиқ')\
#     .replace('Қонунларнинг мураккаблиги', 'Қонунчиликнинг мураккаблиги')\
#     .replace('Қонун устуворлигининг етарлича таъминланмаганлиги', 'Қонунчилик барча учун бир хил ишламайди') \
#     .replace('Божхона божлари юқори', 'Божхона божлари ва тўловлари юқори')

#     old_columns_edit.append(column)

# rename_columns = dict(zip(old_columns, old_columns_edit))
# df_old.rename(columns=rename_columns, inplace=True)
# col_change = '####**23. Сизнингча, солиқ тизимида мавжуд муаммолар нималарда?**'
# df_old[col_change] = df_old[col_change].replace(['Солиқ инспекторлари томонидан олдиндан тўловларни амалга оширишга ундаш', 'Солиқ имтиёзлари сабабли адолат меъзонларининг бузилиши'],
#                                                  ['Солиқларни олдиндан тўлашга (предоплата) ундаш', 'Солиқ имтиёзлари сабабли адолат мезонларининг бузилиши'], regex=True)


# ## old ####**11. Сиз тўлаётган қайси солиқ турининг ставкаси юқори?**
# col_change = '####**11. Сиз тўлаётган қайси солиқ турининг ставкаси юқори?**'
# df_old[col_change] = df_old[col_change].replace(['Ягона солиқ'], ['Ягона ижтимоий солиқ'], regex=True)


# ## old ####**67. Тадбиркорлик фаолиятини тўлиқ қонунийлаштиришга нималар тўсқинлик қилади?**
# col_change = '####**67. Тадбиркорлик фаолиятини тўлиқ қонунийлаштиришга нималар тўсқинлик қилади?**'
# df_old[col_change] = df_old[col_change].replace(['Қонунларнинг мураккаблиги', 'Қонун устуворлигининг етарлича таъминланмаганлиги'],
#                                                 ['Қонунчиликнинг мураккаблиги', 'Қонунчилик барча учун бир хил ишламайди'], regex=True)


# ## old ####**34.1. Импорт операцияларини амалга оширишда қандай муаммоларга дуч келгансиз?**
# col_change = '####**34.1. Импорт операцияларини амалга оширишда қандай муаммоларга дуч келгансиз?**'
# df_old[col_change] = df_old[col_change].replace(['Божхона божлари юқори'], ['Божхона божлари ва тўловлари юқори',], regex=True)



In [155]:
##### fix working years 
col_years = '####**8. Неча йилдан бери фаолият юритасиз?**'
df_old[col_years] = df_old[col_years].map(lambda x: 2023 - int(x) if int(x) > 1900 and int(x) < 2024 else x)


In [156]:
df_trans = pd.read_excel('data\\2023_2024_question_comparison.xlsx')
df_trans = df_trans[(df_trans['name_2023'] != '#N/A') & (~pd.isnull(df_trans['name_2023']))]

In [157]:
for column in df_trans.columns:
    df_trans[column] = df_trans[column].map(lambda x: strip_tags(x).strip() if isinstance(x, str) else x)


In [158]:
def strip_tags_from_columns(df):
    columns = df.columns
    new_columns = []
    for column in columns:
        new_columns.append(strip_tags(column).strip())

    rename_columns = dict(zip(columns, new_columns))
    return df.rename(rename_columns, axis=1)

In [159]:
df_new = strip_tags_from_columns(df_new)
df_old = strip_tags_from_columns(df_old)

count_new = 0
count_old = 0
for i, row in df_trans.iterrows():
    c_new = row['label_2024']
    c_old = row['label_2023']
    if c_new in df_new.columns:
        count_new += 1
    if c_old in df_old.columns:
        count_old += 1

if count_new != count_old:
    raise Exception("Not all columns that shoud match are matching (of old and new dfs).")




In [170]:
def freq_table(df, column):
    freq_column = 'Frequency'
    pct_column = 'Percentage (%)'
    avg_column = 'Average score'
    margins_name = 'Total:'

    ## check multiple question
    columns = df.columns.values.tolist()
    col_idx = columns.index(column)
    if col_idx < len(columns)-1:
        if column == columns[col_idx+1].split('/')[0].strip():
            slash_columns = [c for c in columns if ('/' in c and column in c) and (c != column)]
            unique_answers = [c.split('/', 2)[1] for c in slash_columns]
            df_ctab = pd.DataFrame(columns=[freq_column, pct_column, avg_column], index=unique_answers)
          
            for i, sc in enumerate(slash_columns):
                df_ctab.at[unique_answers[i], freq_column] = int(df[sc].sum())

            total_anwsers = len(df[column].dropna())
            df_ctab[pct_column] = df_ctab[freq_column].divide(total_anwsers).multiply(100).astype('float64').round(2)
            df_ctab[avg_column] = np.nan

            df_ctab.sort_values(by=freq_column, ascending=False, inplace=True)

            total_row = pd.DataFrame(columns=[freq_column, pct_column, avg_column], 
                         index=[margins_name], 
                         data=[[total_anwsers, df_ctab[pct_column].sum(), np.nan]])
            
            df_ctab = pd.concat([df_ctab, total_row], axis=0)


            df_ctab.index.name = column
            return df_ctab.reset_index()

    df[freq_column] = freq_column
    df[pct_column] = pct_column
    
  
    df_ctab_count = pd.crosstab(index=[df[column]], columns=[df[freq_column]], normalize=False, margins=True, margins_name=margins_name)
    df_ctab_pct = pd.crosstab(index=[df[column]], columns=[df[pct_column]], normalize=True, margins=True, margins_name=margins_name)
    df_ctab_pct = df_ctab_pct.multiply(100).round(2)
    df_ctab = pd.merge(df_ctab_count, df_ctab_pct, right_index=True, left_index=True)
    df_ctab.drop(columns=[margins_name+'_x', margins_name+'_y'], inplace=True)
    df_ctab.sort_values(by=[freq_column], ascending=False, inplace=True)
    df_ctab = pd.concat([df_ctab.tail(-1), df_ctab.head(1)], axis=0)
    if is_float_dtype(df[column]) or is_integer_dtype(df[column]):
        df_ctab[avg_column] = df_ctab.index.map(
                                            lambda x: (df_ctab.loc[x, freq_column]* x) / df_ctab[freq_column].sum() 
                                            if x != margins_name
                                            else 1)
        df_ctab.loc[margins_name, avg_column] = df_ctab.head(-1)[avg_column].sum()
        df_ctab[avg_column] = df_ctab[avg_column].round(2)

    else:
        df_ctab[avg_column] = np.nan

    df_ctab.index.name = column
    return df_ctab.reset_index()

    


In [171]:

df_ctab_out = pd.DataFrame()
for i, row in df_trans.iterrows():
    column_new = row['label_2024']
    column_old = row['label_2023']

    df_ctab_new = freq_table(df_new, column_new)
    df_ctab_old = freq_table(df_old, column_old)


    df_ctab = pd.merge(df_ctab_new, 
                       df_ctab_old, 
                       left_on=column_new, 
                       right_on=column_old, 
                       how='outer')
    

    df_ctab = df_ctab.T.reset_index().T

    df_empty = pd.DataFrame(np.nan, index=[0, 1, 2], columns=df_ctab.columns)
    df_ctab = pd.concat([df_ctab, df_empty], axis=0)

    if df_ctab_out.empty:
        df_ctab_out = df_ctab
    else:
        df_ctab_out = pd.concat([df_ctab_out, df_ctab], axis=0)


filename_out = 'out\\outfile.xlsx'
df_ctab_out.to_excel(filename_out, index=False)

12.1. Маҳаллий ҳокимлик фаолиятида қандай салбий (қониқарсиз) ҳолатлар кузатилган? /Коррупция
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
1336    NaN
1337    NaN
1338    0.0
1339    NaN
1340    NaN
Name: 12.1. Маҳаллий ҳокимлик фаолиятида қандай салбий (қониқарсиз) ҳолатлар кузатилган? /Коррупция, Length: 1341, dtype: float64
2.0
12.1. Маҳаллий ҳокимлик фаолиятида қандай салбий (қониқарсиз) ҳолатлар кузатилган? /Ортиқча хужжатлар талаб қилиш (бюрократия)
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
1336    NaN
1337    NaN
1338    0.0
1339    NaN
1340    NaN
Name: 12.1. Маҳаллий ҳокимлик фаолиятида қандай салбий (қониқарсиз) ҳолатлар кузатилган? /Ортиқча хужжатлар талаб қилиш (бюрократия), Length: 1341, dtype: float64
13.0
12.1. Маҳаллий ҳокимлик фаолиятида қандай салбий (қониқарсиз) ҳолатлар кузатилган? /Таниш-билишчилик
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
1336    NaN
1337    NaN
1338    0.0
1339

TypeError: cannot convert the series to <class 'int'>

In [ ]:
wb = openpyxl.load_workbook(filename_out)
worksheet = wb.active
worksheet.column_dimensions['A'].width = 35.57
worksheet.column_dimensions['B'].width = 14.86
worksheet.column_dimensions['C'].width = 14.86
worksheet.column_dimensions['D'].width = 14.86
worksheet.column_dimensions['E'].width = 35.57
worksheet.column_dimensions['F'].width = 14.86
worksheet.column_dimensions['G'].width = 14.86
worksheet.column_dimensions['H'].width = 14.86

worksheet.cell(row=1, column=1).value = 'Тадбиркорлар фаолиятини ўрганиш бўйича сўровнома 2024'
worksheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=8)

clr = Color('ff91d2ff')

for column in range(1, 9):
    for row_idx in range(1, worksheet.max_row):
        if 'Frequency' in str(worksheet.cell(row=row_idx, column=2).value):
            cell = worksheet.cell(row=row_idx, column=column)
            filling = PatternFill(patternType='solid', fgColor=clr)
            cell.fill = filling

    
            alignment = Alignment(horizontal='center', 
                    vertical='center',
                    text_rotation=0,
                    wrap_text=True,
                    shrink_to_fit=True,
                    indent=0) 
            
            cell.alignment = alignment

wb.save(filename=filename_out)

